In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!conda install -y gdown

In [ ]:
!gdown https://drive.google.com/drive/folders/1Yio34S3msz33Vdf-01WNZlUbZaE8dQci?usp=sharing

In [ ]:
from tensorflow.keras.applications import resnet50
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

def create_model(output_shape, lr=0.0001, training=False):
    base_model = resnet50.ResNet50(weights="imagenet",include_top=False,input_tensor=Input(shape=(80,80,3)))
    headModel = base_model.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(output_shape, activation="softmax")(headModel)
    
    model = Model(inputs=base_model.input, outputs=headModel)
    # Training params
    if training:
        for layer in base_model.layers:
            layer.trainable = True
        optimizer = Adam(lr=lr)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])    
        
    return model

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255,
                                   shear_range=0.2,
                                   zoom_range=0.2,
                                   horizontal_flip=True,
                                   validation_split=0.2)

test_datagen = ImageDataGenerator(rescale=1./255)

DIRECTORY = "../input/dataset-recog/dataset_characters"
train_datagen_dir = train_datagen.flow_from_directory(directory=DIRECTORY,target_size=(80,80), batch_size=32,
                                                      subset="training")

validation_datagen_dir = train_datagen.flow_from_directory(directory=DIRECTORY,target_size=(80,80), batch_size=32,
                                                      subset="validation")
model = create_model(output_shape=36, training=True)


In [ ]:
model.fit_generator(train_datagen_dir,epochs=30,validation_data=validation_datagen_dir)

In [ ]:
from tensorflow.keras.models import save_model

save_model(model, "dataset_recog_resnet.h5")

# Mobilenet

In [5]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.applications import EfficientNetB3
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Input
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.models import model_from_json
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import glob
import numpy as np

In [6]:
dataset_paths = glob.glob("../input/dataset-recog/dataset_characters/**/*.jpg")

In [7]:
import os
X=[]
labels=[]

for image_path in dataset_paths:
  label = image_path.split(os.path.sep)[-2]
  image=load_img(image_path,target_size=(80,80))
  image=img_to_array(image)

  X.append(image)
  labels.append(label)

X = np.array(X,dtype="float16")
labels = np.array(labels)

print("[INFO] Find {:d} images with {:d} classes".format(len(X),len(set(labels))))


# perform one-hot encoding on the labels
lb = LabelEncoder()
lb.fit(labels)
labels = lb.transform(labels)
y = to_categorical(labels)


[INFO] Find 37623 images with 36 classes


In [8]:
(trainX, testX, trainY, testY) = train_test_split(X, y, test_size=0.10, stratify=y, random_state=42)


In [9]:
image_gen = ImageDataGenerator(rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest"
                              )

In [10]:
def create_model(lr=1e-4,decay=1e-4/25, training=False,output_shape=y.shape[1]):
    baseModel = EfficientNetB3(weights="imagenet", 
                            include_top=False,
                            input_tensor=Input(shape=(80, 80, 3)))

    headModel = baseModel.output
    headModel = AveragePooling2D(pool_size=(3, 3))(headModel)
    headModel = Flatten(name="flatten")(headModel)
    headModel = Dense(128, activation="relu")(headModel)
    headModel = Dropout(0.5)(headModel)
    headModel = Dense(output_shape, activation="softmax")(headModel)
    
    model = Model(inputs=baseModel.input, outputs=headModel)
    
    if training:
        # define trainable lalyer
        for layer in baseModel.layers:
            layer.trainable = True
        # compile model
        optimizer = Adam(lr=lr, decay = decay)
        model.compile(loss="categorical_crossentropy", optimizer=optimizer,metrics=["accuracy"])    
        
    return model

In [11]:
INIT_LR = 1e-4
EPOCHS = 30

model = create_model(lr=INIT_LR, decay=INIT_LR/EPOCHS,training=True)

43941888/43941136 [==============================] - 1s 0us/step


In [12]:
BATCH_SIZE = 64

my_checkpointer = [
                EarlyStopping(monitor='val_loss', patience=5, verbose=0),
                ModelCheckpoint(filepath="License_character_recognition.h5", verbose=1, save_weights_only=True)
                ]

result = model.fit(image_gen.flow(trainX, trainY, batch_size=BATCH_SIZE), 
                   steps_per_epoch=len(trainX) // BATCH_SIZE, 
                   validation_data=(testX, testY), 
                   validation_steps=len(testX) // BATCH_SIZE, 
                   epochs=EPOCHS, callbacks=my_checkpointer)

Epoch 1/30
529/529 [==============================] - 107s 168ms/step - loss: 2.5356 - accuracy: 0.3430 - val_loss: 0.4000 - val_accuracy: 0.8982

Epoch 00001: saving model to License_character_recognition.h5
Epoch 2/30
529/529 [==============================] - 84s 159ms/step - loss: 0.4113 - accuracy: 0.8940 - val_loss: 0.1942 - val_accuracy: 0.9487

Epoch 00002: saving model to License_character_recognition.h5
Epoch 3/30
529/529 [==============================] - 84s 159ms/step - loss: 0.2527 - accuracy: 0.9321 - val_loss: 0.1523 - val_accuracy: 0.9559

Epoch 00003: saving model to License_character_recognition.h5
Epoch 4/30
529/529 [==============================] - 84s 159ms/step - loss: 0.1877 - accuracy: 0.9476 - val_loss: 0.1177 - val_accuracy: 0.9647

Epoch 00004: saving model to License_character_recognition.h5
Epoch 5/30
529/529 [==============================] - 85s 160ms/step - loss: 0.1608 - accuracy: 0.9536 - val_loss: 0.1108 - val_accuracy: 0.9665

Epoch 00005: saving m

In [13]:

# save model architectur as json file
model_json = model.to_json()
with open("EfficientNet_character_recognition.json", "w") as json_file:
    json_file.write(model_json)